In [13]:
import pandas as pd
import numpy as np
import pymongo
import ast
import datetime
path_data = 'C:/root/Universita/project_dmdv/data/'

In [14]:
#some useful functions

#funzione per la sostituzione sulla base di una regex, la funzione nativa va in errore se il valore è null
def regex_replace(regex, sub, string):
    if pd.isnull(string):
        return None
    else:
        return re.sub(regex, sub, string)

#matching sulla base di una regex, la f nativa va in errore se il valore è mancante
def regex_match(regex, string, reverse = False):
    if pd.isnull(string):
        return None
    else:
        return re.match(regex, string)
    
#is_winner
def get_winner(string):
    if string:
        return 'Winner'
    else:
        return 'Nominee'
    
def replace_na_string(string):
    str_out = string
    if pd.notnull(string):
        if string.replace(' ', '') == '':
            str_out = None
    return str_out

def substring(string, start, end):
    str_out = None
    if pd.notnull(string):
        str_out = string[start:end]
    return str_out

def get_names(lista):
    out_list = []
    for el in lista:
        out_list = out_list + [el['name']]
    out_string = ', '.join(out_list)
    if out_string == '':
        out_string = None
    return out_string

def get_names_list(lista):
    out_list = []
    try:
        len(lista)
    except:
        lista = []
    for el in lista:
        out_list = out_list + [el['name']]
    if len(out_list) == 0:
        out_list = None
    return out_list

def get_names_from_dict(dict_):
    out_string = None
    if  pd.notnull(dict_):
        out_string = dict_['name']
    return out_string

def get_date(date):
    try:
        date_out = pd.to_datetime(date, format = "%Y-%m-%d")
    except:
        date_list = date.split('-')
        if date_list[0][:2] == '10':
            date_list[0] = '19' + date_list[0][-2:]
        elif date_list[0][:2] == '00':
            date_list[0] = '20' + date_list[0][-2:]
        elif len(date_list[0]) > 4:
            date_list[0] = date_list[0][:2] + date_list[0][-2:]
        date_out = pd.to_datetime('-'.join(date_list), format = "%Y-%m-%d")
    return date_out

def convert_dtypes(data):
    out = None
    if pd.notnull(data):
        out = ast.literal_eval(data)
    return out

def mongo_encoding(diz):
    new = {}
    for key, value in diz.items():
        
        #dizionari annidati, funzione ricorsiva
        if isinstance(value, dict):
            value = mongo_encoding(value)
        
        if isinstance(value, np.bool_):
            value = bool(value)
        
        if isinstance(value, np.int64):
            value = int(value)
        
        if isinstance(value, np.float64):
            value = float(value)
        
        if ((not isinstance(value, list)) & (not isinstance(value, dict))):
            if pd.notnull(value):
                new[key] = value
        else:
            new[key] = value

    return new



# IMPORT

### Awards

In [15]:
award_movies = pd.read_pickle(path_data + 'awards/integrated/award_movies_final.df')
award_movies['source'] = 'movie'
award_movies.columns = ['award', 'award_id', 'award_type', 'is_winner', 'person', 'title_id', 'year', 'source']

award_tvs = pd.read_pickle(path_data + 'awards/integrated/award_tvseries_final.df')
award_tvs['source'] = 'tv'

award_people = pd.read_pickle(path_data + 'awards/integrated/award_people_final.df')

In [16]:
award_titles = pd.concat([award_movies, award_tvs])
award_titles = award_titles[pd.notnull(award_titles['title_id'])]

In [17]:
award_titles['diz_awards'] = award_titles.apply(lambda row:
                                                {'award_id' : row['award_id'],'award_type' : row['award_type'], 'award' : row['award'],
                                                 'is_winner' : row['is_winner'], 'year': row['year']
                                                }, axis = 1)

In [18]:
award_title_list = pd.DataFrame(award_titles.groupby('title_id').agg({'diz_awards':lambda x: list(x)}).reset_index())
award_title_list.columns = ['movie_id', 'diz_awards']

In [19]:
award_title_list

,movie_id,diz_awards
0,m-100081,"[{'award_id': 'aw-8621', 'award_type': 'oscar ..."
1,m-100082,"[{'award_id': 'aw-8064', 'award_type': 'oscar ..."
2,m-10010,"[{'award_id': 'aw-1190', 'award_type': 'oscar ..."
3,m-100164,"[{'award_id': 'aw-5092', 'award_type': 'oscar ..."
4,m-100277,"[{'award_id': 'aw-2971', 'award_type': 'oscar ..."
...,...,...
7245,tv-9869,"[{'award_id': 'aw-19106', 'award_type': 'emmy ..."
7246,tv-992,"[{'award_id': 'aw-18288', 'award_type': 'emmy ..."
7247,tv-993,"[{'award_id': 'aw-24461', 'award_type': 'golde..."
7248,tv-9932,"[{'award_id': 'aw-22811', 'award_type': 'golde..."


In [20]:
award_people['diz_awards'] = award_people.apply(lambda row:
                                                {'award_id' : row['award_id'],'award_type' : row['award_type'], 'award' : row['award'],
                                                 'is_winner' : row['is_winner'], 'year': row['year']
                                                }, axis = 1)

In [21]:
award_people_list = pd.DataFrame(award_people.groupby('person_id').agg({'diz_awards':lambda x: list(x)}).reset_index())
award_people_list

,person_id,diz_awards
0,p-1,"[{'award_id': 'aw-5424', 'award_type': 'oscar ..."
1,p-10,"[{'award_id': 'aw-8877', 'award_type': 'oscar ..."
2,p-10001,"[{'award_id': 'aw-3823', 'award_type': 'oscar ..."
3,p-10002,"[{'award_id': 'aw-3234', 'award_type': 'oscar ..."
4,p-1000448,"[{'award_id': 'aw-7287', 'award_type': 'oscar ..."
...,...,...
11119,p-9991,"[{'award_id': 'aw-9713', 'award_type': 'oscar ..."
11120,p-99934,"[{'award_id': 'aw-7169', 'award_type': 'oscar ..."
11121,p-99966,"[{'award_id': 'aw-20110', 'award_type': 'emmy ..."
11122,p-99968,"[{'award_id': 'aw-21625', 'award_type': 'emmy ..."


In [11]:
award_people['list_people'] = award_people.apply(lambda row:
                                                {'person_id' : row['person_id']
                                                }, axis = 1)

In [22]:
award_list_of_people = pd.DataFrame(award_people.groupby('award_id').agg({'person_id':lambda x: list(x)}).reset_index())

In [23]:
award_list_of_people

,award_id,person_id
0,aw-1,[p-12259]
1,aw-10,[p-8254]
2,aw-100,[p-23857]
3,aw-10000,[p-34045]
4,aw-10001,[p-30955]
...,...,...
22364,aw-9995,[p-265916]
22365,aw-9996,[p-121]
22366,aw-9997,[p-119]
22367,aw-9998,[p-8204]


In [30]:
award_out = award_titles.merge(award_list_of_people, how = 'left', on = 'award_id').drop(
    columns = ['diz_awards','person']).rename(columns = {'award_id':'id'})

In [31]:
award_out.head()

,award,id,award_type,is_winner,title_id,year,source,person_id
0,COSTUME DESIGN,aw-9999,oscar award,Nominee,m-362956,2019.0,movie,[p-15542]
1,COSTUME DESIGN,aw-9998,oscar award,Nominee,m-269036,2019.0,movie,[p-8204]
2,CINEMATOGRAPHY,aw-9997,oscar award,Nominee,m-323567,2019.0,movie,[p-119]
3,CINEMATOGRAPHY,aw-9996,oscar award,Winner,m-376054,2019.0,movie,[p-121]
4,CINEMATOGRAPHY,aw-9995,oscar award,Nominee,m-353452,2019.0,movie,[p-265916]


In [33]:
client = pymongo.MongoClient("localhost", 27025)
mongo_tmdb = client.tmdb
mongo_people = mongo_tmdb.awards

In [34]:
awards_insert_list = [mongo_encoding(award_out.iloc[i].to_dict()) for i in range(award_out.shape[0])]
mongo_people.insert_many(awards_insert_list)

### People

In [24]:
people_original = pd.read_json(path_data + 'tmdb/json/tmdb_people.json')
#people.columns = ['person', 'person_id', 'popularity', 'also_known_as']
#people_original['also_known_as'] = [name.strip('[').strip(']').strip("'").split(',') for name in people_original['also_known_as']]
people_original = people_original.applymap(lambda x: replace_na_string(x) if isinstance(x, str) else x)

### Cast - Crew

In [14]:
cast_movies = pd.read_json(path_data + 'tmdb/json/tmdb_movies_cast.json')#[['name', 'movie_id', 'person_id']]
crew_movies = pd.read_json(path_data + 'tmdb/json/tmdb_movies_crew.json')#[['name', 'movie_id', 'person_id']]

In [15]:
cast_movies['diz_cast'] = cast_movies.apply(lambda row:
                              {'person_id' : row['person_id'], 'name' : row['name'], 'character' : row['character'], 'gender' : row['gender'],
                                           'profile_path' : row['profile_path']}, axis = 1)

In [16]:
cast_movies_list = pd.DataFrame(cast_movies.groupby('movie_id').agg({'diz_cast':lambda x: list(x)}).reset_index())
cast_movies_list.columns = ['movie_id', 'diz_cast']

In [17]:
cast_movies['diz_cast_people'] = cast_movies.apply(lambda row:
                              {'movie_id' : row['movie_id'], 'name' : row['name'], 'character' : row['character'], 'gender' : row['gender'],
                                           'profile_path' : row['profile_path']}, axis = 1)
cast_movies_list_people = pd.DataFrame(cast_movies.groupby('person_id').agg({'diz_cast_people':lambda x: list(x)}).reset_index())
cast_movies_list_people.columns = ['person_id', 'diz_cast_people']

In [18]:
crew_movies['diz_crew'] = crew_movies.apply(lambda row:
                              {'person_id' : row['person_id'], 'name' : row['name'], 'job' : row['job'], 'department' : row['department'],
                               'gender' : row['gender'], 'profile_path' : row['profile_path']}, axis = 1)

In [19]:
crew_movies_list = pd.DataFrame(crew_movies.groupby('movie_id').agg({'diz_crew':lambda x: list(x)}).reset_index())
crew_movies_list.columns = ['movie_id', 'diz_crew']

In [23]:
crew_movies_list

,movie_id,diz_crew
0,m-1,"[{'person_id': 'p-29159', 'name': 'Frank R. St..."
1,m-10,"[{'person_id': 'p-1789', 'name': 'Stephen Hopk..."
2,m-100,"[{'person_id': 'p-959', 'name': 'Krzysztof Kie..."
3,m-1000,"[{'person_id': 'p-7614', 'name': 'John A. Amic..."
4,m-10000,"[{'person_id': 'p-149097', 'name': 'Mark Milga..."
...,...,...
437217,m-99995,"[{'person_id': 'p-3191', 'name': 'Suzanne Schi..."
437218,m-99996,"[{'person_id': 'p-660354', 'name': 'Michelle C..."
437219,m-99997,"[{'person_id': 'p-156235', 'name': 'Marie-Thér..."
437220,m-99998,"[{'person_id': 'p-387297', 'name': 'Vanalyne G..."


In [35]:
crew_movies['diz_crew_people'] = crew_movies.apply(lambda row:
                              {'person_id' : row['person_id'], 'name' : row['name'], 'job' : row['job'], 'department' : row['department'],
                               'gender' : row['gender'], 'profile_path' : row['profile_path']}, axis = 1)
crew_movies_list_people = pd.DataFrame(crew_movies.groupby('person_id').agg({'diz_crew_people':lambda x: list(x)}).reset_index())

crew_movies_list_people.columns = ['person_id', 'diz_crew_people']

### Movies

In [24]:
tmdb_movies_complete = pd.read_json(path_data + 'tmdb/json/tmdb_movies.json')

tmdb_movies_complete['original_title'] = pd.core.strings.str_strip(tmdb_movies_complete['original_title'])
tmdb_movies_complete['title'] = pd.core.strings.str_strip(tmdb_movies_complete['title'])

tmdb_movies_complete['production_countries'] = tmdb_movies_complete['production_countries'].apply(
    lambda row: get_names_list(row))
tmdb_movies_complete['spoken_languages'] = tmdb_movies_complete['spoken_languages'].apply(
    lambda row: get_names_list(row))
tmdb_movies_complete['production_companies'] = tmdb_movies_complete['production_companies'].apply(
    lambda row: get_names_list(row))
tmdb_movies_complete['genres'] = tmdb_movies_complete['genres'].apply(
    lambda row: get_names_list(row))


In [28]:
tmdb_movies_complete['belongs_to_collection'] = tmdb_movies_complete['belongs_to_collection'].apply(
    lambda row: get_names_from_dict(row))

#conversione data
tmdb_movies_complete['release_date'] = pd.to_datetime(tmdb_movies_complete['release_date'], format = "%Y-%m-%d")

#def na per le stringhe
tmdb_movies_complete = tmdb_movies_complete.applymap(lambda x: replace_na_string(x) if isinstance(x, str) else x)

#def T/F per i bool
tmdb_movies_complete = tmdb_movies_complete.applymap(lambda x: str(x) if isinstance(x, bool) else x)

In [29]:
tmdb_movies_load = tmdb_movies_complete.copy()[['movie_id', 'title', 'original_title', 'belongs_to_collection', 'release_date',
                                               'genres', 'adult', 'runtime', 'original_language', 'spoken_languages', 
                                               'production_companies', 'production_countries', 'budget', 'revenue',
                                               'tagline', 'overview', 'homepage', 'poster_path', 'id', 'popularity', 'vote_average', 'vote_count']]

In [30]:
tmdb_movies_load = tmdb_movies_load.merge(award_title_list[['movie_id', 'diz_awards']], how = 'left',
                                                  on = 'movie_id').merge(cast_movies_list[['movie_id', 'diz_cast']], how = 'left',
                                                  on = 'movie_id').merge(crew_movies_list[['movie_id', 'diz_crew']], how = 'left',
                                                  on = 'movie_id')
tmdb_movies_load.rename(columns = {'diz_awards' : 'awards', 'diz_crew' : 'crew', 'diz_cast' : 'cast', 'movie_id': 'id', 'id' : 'tmdb_id'}, inplace = True)
tmdb_movies_load.head()

,id,title,original_title,belongs_to_collection,release_date,genres,adult,runtime,original_language,spoken_languages,...,overview,homepage,poster_path,tmdb_id,popularity,vote_average,vote_count,awards,cast,crew
0,m-1,Blondie,Blondie,Blondie Collection,1938-11-30,[Comedy],False,70.0,en,[English],...,Blondie and Dagwood are about to celebrate the...,None,/o6UMTE2LzQdlKVxRnFECPmtQjsJ.jpg,3924,2.620,6.6,4,NaN,"[{'person_id': 'p-29165', 'name': 'Penny Singl...","[{'person_id': 'p-29159', 'name': 'Frank R. St..."
1,m-2,"Peter Voss, Thief of Millions",Der Mann ohne Namen,None,1921-01-01,[Adventure],False,420.0,de,None,...,Der Mann ohne Namen is a German adventure movi...,None,None,6124,0.600,0.0,0,NaN,"[{'person_id': 'p-39518', 'name': 'Harry Liedt...","[{'person_id': 'p-2580', 'name': 'Robert Liebm..."
2,m-3,Love at Twenty,L'amour à vingt ans,None,1962-06-22,"[Romance, Drama]",False,110.0,fr,"[Français, 日本語, Italiano, Polski, Deutsch]",...,Love at Twenty unites five directors from five...,None,/sU4o25RZullugxoXZstXKN8bPPF.jpg,8773,3.430,6.9,29,NaN,"[{'person_id': 'p-1424', 'name': 'Jean-Pierre ...","[{'person_id': 'p-1421', 'name': 'François Tru..."
3,m-4,Jatts in Golmaal,Jatts in Golmaal,None,2003-02-21,[Comedy],False,NaN,pa,None,...,Jatts in Golmal is an Comedy based movie. In w...,None,/g5epyphS4WqmwJDIu14EL20gxuc.jpg,21624,0.600,0.0,0,NaN,NaN,NaN
4,m-5,New World Disorder 9: Never Enough,New World Disorder 9: Never Enough,New World Disorder,2008-12-08,None,False,69.0,en,[English],...,Gee Atherton ripping the Worlds course the day...,http://www.nwdfilms.com,/okQY6jVmRU19CUbYPUZC77K3XBE.jpg,25449,0.921,4.5,2,NaN,"[{'person_id': 'p-69986', 'name': 'Darren Berr...","[{'person_id': 'p-94540', 'name': 'Derek Weste..."


In [66]:
client = pymongo.MongoClient("localhost", 27025)
mongo_tmdb = client.tmdb
mongo_movies = mongo_tmdb.movies

In [92]:
#CODICE USATO PER LANCIARE A PEZZI E INTERROGARE PER I GRAFICI
#shape = tmdb_movies_load.shape[0]
#value = np.round(shape/4)
#tresholds = [i for i in range(4)]
#list_of_ranges = [[int(t*value), int((t+1)*value)] for t in tresholds]
#list_of_ranges[3][1] = shape
#r = list_of_ranges[3]
#movies_insert_list = [mongo_encoding(tmdb_movies_load.iloc[i].to_dict()) for i in range(r[0], r[1])]

In [81]:
movies_insert_list = [mongo_encoding(tmdb_movies_load.iloc[i].to_dict()) for i in range(tmdb_movies_load.shape[0])]
mongo_movies.insert_many(movies_insert_list)

### TV Series

In [10]:
tv_series_original = pd.read_json(path_data + 'tmdb/json/tmdb_tvs.json')#[
#    ['first_air_date','id', 'last_air_date', 'name', 'original_name', 'popularity', 'series_id']]
#tv_series_original.columns = ['first_air_date','id', 'last_air_date', 'title', 'original_title', 'popularity', 'title_id']


tv_series_original['first_air_date'] = tv_series_original['first_air_date'].apply(get_date)
tv_series_original['last_air_date'] = tv_series_original['last_air_date'].apply(get_date)

#def na per le stringhe
tv_series_original = tv_series_original.applymap(lambda x: replace_na_string(x) if isinstance(x, str) else x)
tv_series_original['created_by'] = tv_series_original['created_by'].apply(
    lambda row: get_names_list(row))

##### Seasons

In [14]:
tv_seasons_complete = pd.read_json(path_data + 'tmdb/json/tmdb_tv_seasons.json')
tv_seasons_complete.columns = ['_id', 'air_date', 'episodes', 'name', 'overview', 'id',
       'poster_path', 'season_number', 'title_id', 'my_n_season',
       'year_title']
#def na per le stringhe
tv_seasons_complete = tv_seasons_complete.applymap(lambda x: replace_na_string(x) if isinstance(x, str) else x)

In [16]:
tv_seasons_complete['diz_seasons'] = tv_seasons_complete.apply(lambda row:
                              {'season_number' : row['season_number'], 'name' : row['name'], 'air_date' : row['air_date'], 'poster_path' : row['poster_path'],
                                           'overview' : row['overview'], 'episodes' : row['episodes']}, axis = 1)

In [19]:
tv_seasons = pd.DataFrame(tv_seasons_complete.groupby('title_id').agg({'diz_seasons':lambda x: list(x)}).reset_index())
tv_seasons.columns = ['series_id', 'diz_seasons']

##### Cast & Crew

In [24]:
cast_tvs = pd.read_json(path_data + 'tmdb/json/tmdb_tvs_cast.json')#[['name', 'movie_id', 'person_id']]
crew_tvs = pd.read_json(path_data + 'tmdb/json/tmdb_tvs_crew.json')#[['name', 'movie_id', 'person_id']]

In [25]:
cast_tvs['diz_cast'] = cast_tvs.apply(lambda row:
                              {'person_id' : row['person_id'], 'name' : row['name'], 'character' : row['character'], 'gender' : row['gender'],
                                           'profile_path' : row['profile_path']}, axis = 1)

In [26]:
cast_tvs_list = pd.DataFrame(cast_tvs.groupby('series_id').agg({'diz_cast':lambda x: list(x)}).reset_index())
cast_tvs_list.columns = ['series_id', 'diz_cast']

In [27]:
cast_tvs['diz_cast_people'] = cast_tvs.apply(lambda row:
                              {'series_id' : row['series_id'], 'name' : row['name'], 'character' : row['character'], 'gender' : row['gender'],
                                           'profile_path' : row['profile_path']}, axis = 1)
cast_tvs_list_people = pd.DataFrame(cast_tvs.groupby('person_id').agg({'diz_cast_people':lambda x: list(x)}).reset_index())
cast_tvs_list_people.columns = ['person_id', 'diz_cast_people']

In [28]:
crew_tvs['diz_crew'] = crew_tvs.apply(lambda row:
                              {'person_id' : row['person_id'], 'name' : row['name'], 'job' : row['job'], 'department' : row['department'],
                               'gender' : row['gender'], 'profile_path' : row['profile_path']}, axis = 1)

In [29]:
crew_tvs_list = pd.DataFrame(crew_tvs.groupby('series_id').agg({'diz_crew':lambda x: list(x)}).reset_index())
crew_tvs_list.columns = ['series_id', 'diz_crew']

In [30]:
crew_tvs['diz_crew_people'] = crew_tvs.apply(lambda row:
                              {'series_id' : row['series_id'], 'name' : row['name'], 'job' : row['job'], 'department' : row['department'],
                               'gender' : row['gender'], 'profile_path' : row['profile_path']}, axis = 1)
crew_tvs_list_people = pd.DataFrame(crew_tvs.groupby('person_id').agg({'diz_crew_people':lambda x: list(x)}).reset_index())
crew_tvs_list_people.columns = ['person_id', 'diz_crew_people']

##### Load Data

In [31]:
tmdb_tvs_load = tv_series_original.copy()[
    ['series_id', 'name', 'original_name', 'genres', 'type', 'in_production', 'status', 'episode_run_time', 'number_of_episodes', 'number_of_seasons',
     'last_air_date', 'last_episode_to_air', 'next_episode_to_air', 'first_air_date', 'languages', 'original_language', 'origin_country', 'created_by', 
     'production_companies','networks', 'overview', 'homepage', 'poster_path', 'id', 'popularity', 'vote_average', 'vote_count']]

In [32]:
tmdb_tvs_load = tmdb_tvs_load.merge(award_title_list[['movie_id', 'diz_awards']], how = 'left',
                                                  left_on = 'series_id', right_on = 'movie_id').merge(cast_tvs_list[['series_id', 'diz_cast']], how = 'left',
                                                  on = 'series_id').merge(crew_tvs_list[['series_id', 'diz_crew']], how = 'left',
                                                  on = 'series_id').merge(tv_seasons, how = 'left', on = 'series_id')
tmdb_tvs_load.rename(columns = {'diz_awards' : 'awards', 'diz_crew' : 'crew', 'diz_cast' : 'cast', 'series_id': 'id', 'id' : 'tmdb_id'}, inplace = True)
tmdb_tvs_load.head()

,id,name,original_name,genres,type,in_production,status,episode_run_time,number_of_episodes,number_of_seasons,...,poster_path,tmdb_id,popularity,vote_average,vote_count,movie_id,awards,cast,crew,diz_seasons
0,tv-1,Pride,プライド,"[{'id': 18, 'name': 'Drama'}]",Scripted,False,Ended,[60],11.0,1,...,/9Ub2BwnLYKoiSaQF93ItyXriCon.jpg,1,8.111,8.2,6,NaN,NaN,"[{'person_id': 'p-11376', 'name': 'Takuya Kimu...","[{'person_id': 'p-632417', 'name': 'Shinji Noj...","[{'season_number': 1, 'name': 'Season 1', 'air..."
1,tv-2,Clerks: The Animated Series,Clerks: The Animated Series,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Scripted,False,Ended,"[30, 22]",6.0,1,...,/xunXvzFlkf1GGgMkCySA9CCFumB.jpg,2,9.721,6.8,54,NaN,NaN,"[{'person_id': 'p-20691', 'name': 'Brian O'Hal...","[{'person_id': 'p-502302', 'name': 'Richard J....","[{'season_number': 0, 'name': 'Specials', 'air..."
2,tv-3,The Message,The Message,"[{'id': 35, 'name': 'Comedy'}]",Scripted,True,Ended,"[25, 30]",6.0,1,...,/wK9h8FwbmOWlMyW6fT2C6yFPvSu.jpg,3,1.220,9.5,1,NaN,NaN,"[{'person_id': 'p-34242', 'name': 'Matt Wilkin...",NaN,"[{'season_number': 1, 'name': 'Season 1', 'air..."
3,tv-4,The Amazing Mrs Pritchard,The Amazing Mrs Pritchard,"[{'id': 18, 'name': 'Drama'}]",Scripted,False,Ended,[60],6.0,1,...,/jEWIXx9Bv35t15yfSODJ55JrxJM.jpg,4,3.214,8.5,1,NaN,NaN,"[{'person_id': 'p-29714', 'name': 'Jane Horroc...","[{'person_id': 'p-322774', 'name': 'Sally Wain...","[{'season_number': 1, 'name': 'Season 1', 'air..."
4,tv-5,La Job,La Job,"[{'id': 35, 'name': 'Comedy'}]",Scripted,False,Ended,[30],12.0,1,...,None,5,7.208,0.6,108,NaN,NaN,"[{'person_id': 'p-167033', 'name': 'Sophie Cad...","[{'person_id': 'p-322776', 'name': 'Anne-Marie...","[{'season_number': 0, 'name': 'Specials', 'air..."


In [33]:
client = pymongo.MongoClient("localhost", 27025)
mongo_tmdb = client.tmdb
mongo_series = mongo_tmdb.tvseries

In [40]:
series_insert_list = [mongo_encoding(tmdb_tvs_load.iloc[i].to_dict()) for i in range(0, tmdb_tvs_load.shape[0])]

In [36]:
mongo_series.insert_many(series_insert_list)

### People

In [37]:
people_original = pd.read_json(path_data + 'tmdb/json/tmdb_people.json')
people_original = people_original.applymap(lambda x: replace_na_string(x) if isinstance(x, str) else x)

In [38]:
people_original_load = people_original.merge(award_people_list[['person_id', 'diz_awards']], how = 'left',
                                                  left_on = 'series_id', right_on = 'person_id').merge(cast_tvs_list_people[['person_id', 'diz_cast_people']], how = 'left',
                                                  on = 'person_id').merge(crew_tvs_list_people[['person_id', 'diz_crew_people']], how = 'left',
                                                  on = 'person_id')
people_original_load.rename(columns = {'diz_awards' : 'awards', 'diz_crew_people' : 'crew', 'diz_cast_people' : 'cast',
                                       'person_id': 'id', 'id' : 'tmdb_id'}, inplace = True)
people_original_load.head()

In [39]:
client = pymongo.MongoClient("localhost", 27025)
mongo_tmdb = client.tmdb
mongo_people = mongo_tmdb.people

In [40]:
people_insert_list = [mongo_encoding(people_original_load.iloc[i].to_dict()) for i in range(0, people_original_load.shape[0])]

In [41]:
mongo_people.insert_many(people_insert_list)